## 데이터 다운로드
--- 
아래 셀을 실행시켜 데이터를 colab에 불러옵니다.  
셀 실행은 <br>Windows: `Ctrl` + `Enter`<br>
Mac: `Shift` + `Enter` <br>
를 이용해 실행 시킬 수 있습니다. 

In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3l6g8j3'

import zipfile
with zipfile.ZipFile('3l6g8j3', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-02-07 13:50:37--  https://bit.ly/3l6g8j3
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://docs.google.com/uc?export=download&id=1n-cTWLp385P-DmmNuMAg6zJDw8MNAPwu [following]
--2022-02-07 13:50:37--  https://docs.google.com/uc?export=download&id=1n-cTWLp385P-DmmNuMAg6zJDw8MNAPwu
Resolving docs.google.com (docs.google.com)... 142.251.45.14, 2607:f8b0:4004:83e::200e
Connecting to docs.google.com (docs.google.com)|142.251.45.14|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9bog60o9b1u6nnlu5olckqqtilcuv267/1644241800000/17946651057176172524/*/1n-cTWLp385P-DmmNuMAg6zJDw8MNAPwu?e=download [following]
--2022-02-07 13:50:40--  https://doc-0g-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deff

In [2]:
import pandas as pd

train = pd.read_csv('data/train.csv',usecols = ['category','data'])
test = pd.read_csv('data/test.csv',usecols = ['data'])

In [3]:
# train,test 데이터에서  '\\n' 단어 제거

train['data'] = train['data'].apply(lambda x : str(x).replace('\\n',' ')) # 텍스트 데이터에서 '\\n' 문자열을 ' '로 변경
test['data'] = test['data'].apply(lambda x : str(x).replace('\\n',' ')) # 텍스트 데이터에서 '\\n' 문자열을 ' '로 변경

In [4]:
# 라이브러리 로딩
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF으로 train 데이터를 피처 벡터화 변환 수행

vect = TfidfVectorizer()
vect.fit(train['data'])
train_x =  vect.transform(train['data'])

print('train data size', train_x.shape)

train data size (40000, 697226)


## 실습
---


In [8]:
# LightGBM을 이용해 학습 및 검증 진행
from lightgbm import LGBMClassifier

#모델 정의 
model = LGBMClassifier()

In [9]:
from sklearn.model_selection import GridSearchCV

# parameter 들을 dictionary 형태로 설정
# max_depth 와 min_samples_split 를 튜닝
parameters = {'max_depth': [1,2,3],
              'min_samples_split':[2,3]}


# GridSearchCV 를 이용해 하이퍼 파리미터 튜닝(cv = 5 로 지정)


Grid = GridSearchCV(model, param_grid=parameters, cv=5)

# fit() 메소드를 이용해 학습
Grid.fit(train_x,train['category'])

estimator = Grid.best_estimator_
estimator

LGBMClassifier(max_depth=3, min_samples_split=2)

In [10]:
print('GridSearchCV 최적 파라미터:', Grid.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(Grid.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.7309
